In [ ]:
!pip install ..
!pip install datasets
!pip install pycocotools
!pip install bitsandbytes

In [2]:
import torch
from rudalle import get_vae
from rudalle.utils import seed_everything

import sys
sys.path.insert(0, 'ru-dolph')

import os
from glob import glob
from os.path import join
from datetime import datetime

import torch
import bitsandbytes as bnb
from torch.utils.data import Dataset, DataLoader
import torchvision
import transformers
import more_itertools
import numpy as np
import pandas as pd
import random
import json
from PIL import Image, ImageDraw
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm.auto import tqdm
from einops import rearrange
import youtokentome as yttm

import torchvision
from rudalle.image_prompts import ImagePrompts
from rudolph.model import get_rudolph_model, ruDolphModel, FP16Module
from rudolph import utils
from rudolph.model.utils import get_attention_mask
from rudalle import get_tokenizer, get_vae
from rudolph.api import ruDolphApi

import datasets
import random
from pycocotools.coco import COCO
import json
from PIL import Image

from typing import Any, Callable, Optional, Tuple, List


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [3]:
device = 'cuda'

In [4]:
torch.cuda.empty_cache()

In [5]:
torch.manual_seed(45)
np.random.seed(45)
random.seed(45)

In [6]:
seed_everything(45)

In [7]:
model = get_rudolph_model('1.3B', pretrained=True, fp16=True, device='cuda')

NEW VERSION!!!!!!


/home/jovyan/.imgenv-rudolph-obj-detection-0/lib/python3.7/site-packages/huggingface_hub/file_download.py:632: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


In [8]:
tokenizer = get_tokenizer()
vae = get_vae(dwt=False).to(device)

tokenizer --> ready
Working with z of shape (1, 256, 32, 32) = 262144 dimensions.
vae --> ready


In [30]:
class Args():
    def __init__(self, model, checkpoint_path):
        self.device = model.get_param('device')
        self.l_text_seq_length = model.get_param('l_text_seq_length')
        self.r_text_seq_length = model.get_param('r_text_seq_length')
        self.image_tokens_per_dim = model.get_param('image_tokens_per_dim')
        self.image_seq_length = model.get_param('image_seq_length')
        self.epochs = 1
        self.save_path= checkpoint_path
        self.model_name = 'rudolph_object_detection_grounding_neg_sampling_coco_vg_nmsn_'
        self.save_every = 10000
        self.bs = 1
        self.clip = 1.0
        self.lr = 2e-5
        self.wandb = False
        self.lt_loss_weight = 0.01
        self.img_loss_weight = 0.01
        self.rt_loss_weight = 7
        self.loc_loss_weight = 0.05
        self.category_weight = 0
        self.bin_size = 8
        self.nms_loss_weight=0.1
        self.categories_num=2
        self.iou_threshold=0.3
        self.conf_num_bins=10
        self.ob_seq_len = 5
        self.conf_loss_weight=0
        self.cat_idx=4
        self.conf_idx=None
        self.loc_len=4
        self.image_size = self.image_tokens_per_dim * self.bin_size
        
checkpoint_path = '../model/checkpoints/'
args = Args(model, checkpoint_path)
if not os.path.exists(args.save_path):
    os.makedirs(args.save_path)

In [29]:
class Args():
    def __init__(self, model, checkpoint_path):
        self.device = model.get_param('device')
        self.l_text_seq_length = model.get_param('l_text_seq_length')
        self.r_text_seq_length = model.get_param('r_text_seq_length')
        self.image_tokens_per_dim = model.get_param('image_tokens_per_dim')
        self.image_seq_length = model.get_param('image_seq_length')
        self.epochs = 1
        self.save_path= checkpoint_path
        self.model_name = 'rudolph_object_detection_grounding_neg_sampling_coco_vg_nmsn_conf_'
        self.save_every = 10000
        self.bs = 1
        self.clip = 1.0
        self.lr = 2e-5
        self.wandb = False
        self.lt_loss_weight = 0.01
        self.img_loss_weight = 0.01
        self.rt_loss_weight = 7
        self.loc_loss_weight = 0.05
        self.category_weight = 0
        self.bin_size = 8
        self.nms_loss_weight=0.1
        self.categories_num=2
        self.iou_threshold=0.3
        self.conf_num_bins=10
        self.ob_seq_len = 6
        self.conf_loss_weight=0.
        self.cat_idx=4
        self.conf_idx=5
        self.loc_len=4
        self.image_size = self.image_tokens_per_dim * self.bin_size
        
checkpoint_path = '../model/checkpoints/'
args = Args(model, checkpoint_path)
if not os.path.exists(args.save_path):
    os.makedirs(args.save_path)

In [10]:
SPC_TOKENS = {
    '<LT_UNK>': 16384,
    '<RT_UNK>': 16385,
    '<LT_T2I>': 16386,
    '<LT_I2T>': 16387,
    '<LT_T2T>': 16388,
    '<RT_I2T>': 16389,
    
    '<LT_TQA>': 16390,
    '<RT_TQA>': 16391,
    '<LT_ODG>': 16392,
    '<RT_ODG>': 16393,
}

In [11]:
for n in range(args.image_tokens_per_dim):
    n_token = SPC_TOKENS['<RT_ODG>']+1+n
    SPC_TOKENS['<RT_OD'+str(n)+'>'] = n_token
last_spc_token = n_token

SPC_TOKENS['<RT_NB>']= last_spc_token+1
SPC_TOKENS['<RT_NL>']= last_spc_token+2
last_spc_token = SPC_TOKENS['<RT_NL>']

In [12]:
for n in range(args.conf_num_bins):
    n_token = last_spc_token+1+n
    SPC_TOKENS['<RT_OD_CONF'+str(n)+'>'] = n_token
last_spc_token = n_token

In [13]:
SPC_TOKENS['<RT_OD_CONF_UNK>'] = last_spc_token+1

In [18]:
image_transform = T.Compose([
    T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
    T.ToTensor(),
    T.Resize((args.image_size, args.image_size))
    ])

In [14]:
exclude_list = np.load('../../data/coco_exclude.npy')

In [15]:
class CocoVisualGenomeDataset(Dataset):
    spc_id = -1

    def __init__(
        self,
        tokenizer,
        rus_categories_list,
        coco_exclude_list,
        coco_root: str,
        coco_ann_file: str,
        vg_annotation_path: str,
        vg_images_root: str,
        args,
        image_transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ):
        super().__init__()
        
        self.coco_root = coco_root
        
        self.args = args
        
        self.image_size = args.image_size
        self.bin_size = args.bin_size
        self.r_text_seq_length = args.r_text_seq_length
        self.l_text_seq_length = args.l_text_seq_length
        self.image_tokens_per_dim = args.image_tokens_per_dim
        
        self.image_transform = image_transform
        
        self.tokenizer = tokenizer
        self.tokenizer_eos_id = tokenizer.eos_id
        self.tokenizer_bos_id = tokenizer.bos_id
        self.spc_tokens = SPC_TOKENS
        
        self.data = []
        
        text_prompt = 'найди на картинке все '

        self.coco = COCO(coco_ann_file)
        self.idxs = list(sorted(self.coco.imgs.keys()))
        for ex_img in coco_exclude_list:
            self.idxs.remove(ex_img)
         
        
        categories = self.coco.dataset['categories']
        cat_idxs = list([cat['id'] for cat in categories])
        self.category_encoding_map = {}
        for i, cat in enumerate(categories):
            self.category_encoding_map[cat['id']] = self.encode_text(text=text_prompt+rus_categories_list[i],
                                                                text_seq_length=args.l_text_seq_length)
        
        
        for idx in self.idxs:
            anns = self.coco.loadAnns(self.coco.getAnnIds(idx))
            anns_df = pd.DataFrame(anns)
            if len(anns_df.columns)>0:
                cat_id_set = set(anns_df['category_id'])
                for cat_id in cat_id_set:
                    coco_image = self.coco.loadImgs(idx)[0]["file_name"]
                    new_sample = {'encoded_left':self.category_encoding_map[cat_id],
                                 'bboxes':anns_df[anns_df['category_id']==cat_id]['bbox'].tolist(),
                                'image_address':os.path.join(self.coco_root, coco_image)}
                    self.data.append(new_sample)
        
        self.vg_images_root = vg_images_root
        with open(vg_annotation_path, 'rb') as fj:
            vg_annotation = json.load(fj)
        
        for image_name in vg_annotation.keys():
            if image_name in os.listdir(self.vg_images_root):
                ann_img = vg_annotation[image_name]
                for phrase in ann_img.keys():
                    new_sample = {'encoded_left':self.encode_text(text=text_prompt+phrase,
                                                                text_seq_length=args.l_text_seq_length),
                                 'bboxes':[ann_img[phrase][0]],
                                'image_address':os.path.join(self.vg_images_root, image_name)}
                    self.data.append(new_sample)
            
    def _load_image(self, image_address) -> Image.Image:
        return Image.open(image_address)
    
    def resize_bbox(self, bbox, img_size):
        
        bbox_x0 = bbox[0]
        bbox_y0 = bbox[1]
        bbox_x1 = bbox_x0+bbox[2]
        bbox_y1 = bbox_y0+bbox[3]
        
        img_w = img_size[0]
        img_h = img_size[1]
        
        max_wh = max(img_w,img_h)

        resized_x0 = int(bbox_x0*self.image_size/img_w)
        resized_y0 = int(bbox_y0*self.image_size/img_h)
        resized_x1 = int(bbox_x1*self.image_size/img_w)
        resized_y1 = int(bbox_y1*self.image_size/img_h)
        return [resized_x0,resized_y0,resized_x1, resized_y1]
    
    def encode_bbox(self, resized_bbox):
        
        bbox_x0_bin = resized_bbox[0]//self.bin_size
        bbox_y0_bin = resized_bbox[1]//self.bin_size
        bbox_x1_bin = min(resized_bbox[2]//self.bin_size, self.image_tokens_per_dim-1)
        bbox_y1_bin = min(resized_bbox[3]//self.bin_size, self.image_tokens_per_dim-1)
        
        new_bbox_tokens = [SPC_TOKENS['<RT_OD'+str(bbox_x0_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_y0_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_x1_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_y1_bin)+'>']]
        
        return new_bbox_tokens
    
    def create_new_box(self):
        bbox_x0_bin = random.randint(0, self.args.image_tokens_per_dim-3)
        bbox_y0_bin = random.randint(0, self.args.image_tokens_per_dim-3)
        bbox_x1_bin = random.randint(bbox_x0_bin, self.args.image_tokens_per_dim-1)
        bbox_y1_bin = random.randint(bbox_y0_bin, self.args.image_tokens_per_dim-1)
        new_bbox_tokens = [SPC_TOKENS['<RT_OD'+str(bbox_x0_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_y0_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_x1_bin)+'>'],
                           SPC_TOKENS['<RT_OD'+str(bbox_y1_bin)+'>'],
                           SPC_TOKENS['<RT_NL>'],
                           SPC_TOKENS['<RT_OD_CONF_UNK>']
                          ]
        return new_bbox_tokens
    
    def encode_target(self, bboxes, img_size):
        encoded_target = [self.tokenizer_bos_id, SPC_TOKENS['<RT_ODG>']]
        ground_truth = []
        
        for i,bbox in enumerate(bboxes):
            
            resized_bbox = self.resize_bbox(bbox, img_size)
            encoded_bbox = self.encode_bbox(resized_bbox)
            
            ground_truth += resized_bbox
            ground_truth += [0]
            
            for loc_token in encoded_bbox:
                encoded_target.append(loc_token)
            encoded_target.append(SPC_TOKENS['<RT_NB>'])
            encoded_target.append(SPC_TOKENS['<RT_OD_CONF_UNK>'])
        for k in range((self.r_text_seq_length-2-len(encoded_target))//args.ob_seq_len+1):
            encoded_target += self.create_new_box()
        
        return torch.Tensor(encoded_target), torch.Tensor(ground_truth)
    
    def encode_text(self, text, text_seq_length, add_special = True):
        tokens = self.tokenizer.tokenizer.encode([text], output_type=yttm.OutputType.ID)[0]
        bos = [self.tokenizer.bos_id]
        if add_special:
            bos.append(self.spc_id)
        tokens = bos + tokens + [self.tokenizer.eos_id]
        return self.tokenizer.prepare_tokens(tokens, text_seq_length)
        
    def __getitem__(self, index):
        
        sample = self.data[index]
        left_encoded_text = sample['encoded_left']
        bboxes = sample['bboxes']
        image_address = sample['image_address']
        
        image = self._load_image(image_address)
        img_size = image.size
        transformed_image = self.image_transform(image)
        
        left_special_token = '<LT_ODG>'
        right_special_token = '<RT_ODG>'
        
        left_encoded_text[torch.where(left_encoded_text == self.spc_id)] = self.spc_tokens[left_special_token]
        
        encoded_target, ground_truth = self.encode_target(bboxes, img_size)

        return left_encoded_text, transformed_image, encoded_target, ground_truth
    
    def __len__(self):
        return len(self.data)

In [16]:
cat_rus_names = ["человек","велосипед","автомобиль","мотоцикл","самолёт","автобус","поезд","грузовик","лодка",
                 "светофор","гидрант","знак стоп","парковочный счетчик","скамейка","птица","кошка","собака",
                "лошадь","овца","корова","слон","медведь","зебра","жираф","рюкзак","зонт","сумочка","галстук",
                "чемодан","фрисби","лыжи","сноуборд","спортивный мяч","воздушный змей","бейсбольная бита",
                 "бейсбольная перчатка","скейтборд","доска для сёрфинга","теннисная ракетка","бутылка","бокал для вина",
                 "чашка","вилка","нож","ложка","миска","банан","яблоко","сэндвич","апельсин","брокколи","морковь",
                 "ход-дог","пицца","пончик","торт","стул","диван","растение в горшке","кровать","обеденный стол","туалет","телевизор",
                 "ноутбук","компьютерная мышь","пульт","клавиатура","мобильный телефон","микроволновка","духовка",
                 "тостер","раковина","холодильник","книга","часы","ваза","ножницы","плюшевый мишка","фен","зубная щётка"]

In [19]:
train_dataset = CocoVisualGenomeDataset(coco_root=r'../../data/train2017',
                             coco_ann_file=r'../../data/annotations/instances_train2017.json',
                             image_transform=image_transform,
                             tokenizer=tokenizer,
                             args = args,
                             coco_exclude_list=exclude_list,
                             rus_categories_list=cat_rus_names,
                                      vg_annotation_path=r'../../data/vg_intersection_rus_train.json',
                                      vg_images_root = r'../../data/VG_100K'
                            )

loading annotations into memory...
Done (t=13.16s)
creating index...
index created!


In [20]:
def decode_text(tokenizer, encoded, ignore_ids):
    return tokenizer.tokenizer.decode(encoded.cpu().numpy().tolist(), ignore_ids=ignore_ids)[0]

In [21]:
ignore_ids = [
    tokenizer.eos_id, tokenizer.bos_id, tokenizer.unk_id, tokenizer.pad_id,
    -1, *list(SPC_TOKENS.values())
]

In [22]:
train_dataloader = DataLoader(train_dataset, batch_size=args.bs, shuffle=True, drop_last=True)

In [23]:
left, img ,tgt, _ = next(iter(train_dataloader))

In [25]:
decode_text(tokenizer, left, ignore_ids)

'найди на картинке все банан'

In [24]:
def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False,
):
    for name, p in model.module.named_parameters():
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
    return model

In [25]:
def load_checkpoint(model, optimizer, filename='checkpoint.pt'):
    if os.path.isfile(filename):
        print("=> loading checkpoint '{}'".format(filename))
        checkpoint = torch.load(filename)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
                  .format(filename, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(filename))

    return model, optimizer

In [26]:
model.train()
optimizer = bnb.optim.Adam8bit(model.parameters(), lr=args.lr)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=args.lr, final_div_factor=500, 
    steps_per_epoch=len(train_dataloader), epochs=args.epochs
)

In [27]:
def decode_right(right_tokens, length=None, scores=True, ob_seq_len=5, cat_idx=4, loc_len=4):
    boxes = torch.zeros((length, loc_len))
    labels = torch.ones(length)
    scores = torch.zeros(length)
    
    for k in range(length):
        boxes[k,:] = right_tokens[k*ob_seq_len:k*ob_seq_len+cat_idx].unsqueeze(0).cpu()
        scores[k] = 1
        
    boxes -= SPC_TOKENS['<RT_OD0>']*torch.ones((length, loc_len))
    boxes *= args.bin_size

    return [{'boxes':boxes, 'labels':labels, 'scores':scores}]

def decode_ground_truth(ground_truth, length=None, scores=True, ob_seq_len=5, cat_idx=4, ):
    boxes = torch.zeros((length, 4))
    labels = (SPC_TOKENS['<RT_NL>']-SPC_TOKENS['<RT_NB>'])*torch.ones(length)
    scores = torch.zeros(length)
    
    for k in range(len(ground_truth)//ob_seq_len):
        labels[k] = ground_truth[k*ob_seq_len+cat_idx].unsqueeze(0).cpu()
        boxes[k,:] = ground_truth[k*ob_seq_len:k*ob_seq_len+cat_idx].unsqueeze(0).cpu()
        scores[k] = 1

    return [{'boxes':boxes, 'labels':labels, 'scores':scores}]

In [31]:
def train(model, optimizer, scheduler, args: Args, train_dataloader, from_checkpoints = False):
    """
      args - arguments for training

      train_dataloader - VisualGenomeDataset
      """
    loss_logs = []
    encoded_right_text = None
    #try:
    t_steps = len(train_dataloader)*args.epochs
    progress = tqdm(total = t_steps, desc='🦌🦌🦌finetuning process🦌🦌🦌')

    save_counter = 0
    start_epoch = 0

    if from_checkpoints:
        save_counter = max([int(f.split('_')[-1].split('.')[0]) for f in os.listdir(args.save_path) if f.startswith(args.model_name)])
        model, optimizer, start_epoch, loss_logs = load_checkpoint(model, optimizer, loss_logs, filename=os.path.join(args.save_path,f"{args.model_name}state_{save_counter}.pt"))
        start_epoch -= 1

    for epoch in range(start_epoch, args.epochs):

        for encoded_left_text, images, encoded_right_text, _ in train_dataloader: #, prompt, image, locs
            
            bs_text = encoded_right_text.shape[0]

            images = images.to(device)

            image_seq_length = args.image_tokens_per_dim ** 2
            total_seq_length = args.l_text_seq_length + image_seq_length + args.r_text_seq_length
            encoded_images = vae.get_codebook_indices(images, disable_gumbel_softmax=True)

            attention_mask_text = get_attention_mask(bs_text, args.l_text_seq_length,
                                             args.image_tokens_per_dim, 
                                             args.r_text_seq_length, args.device)
            input_ids_text = torch.cat((encoded_left_text.to(args.device).int(),
                                encoded_images.int(),
                                encoded_right_text.to(args.device).int()), dim=1)
            if input_ids_text.size(1)==args.l_text_seq_length+args.image_tokens_per_dim+args.r_text_seq_length:
                
                loss, loss_values = model.forward(input_ids_text.long(), attention_mask_text, 
                                                                 lt_loss_weight=args.lt_loss_weight,
                                                                 img_loss_weight=args.img_loss_weight, 
                                                                 rt_loss_weight=args.rt_loss_weight,
                                                  return_loss=True,
                                                  category_weight=args.category_weight,
                                                  fake_category_token = SPC_TOKENS['<RT_NL>'],
                                                  loc_loss_weight=args.loc_loss_weight,
                                                  zero_loc_token = SPC_TOKENS['<RT_OD0>'],
                                                  nms_loss_weight=args.nms_loss_weight,
                                                  categories_num=args.categories_num,
                                                  iou_threshold=args.iou_threshold,
                                                  conf_loss_weight=args.conf_loss_weight,
                                                  iou_bin_tokens=[SPC_TOKENS['<RT_OD_CONF'+str(n)+'>'] for n in range(args.conf_num_bins)],
                                                  cat_idx=args.cat_idx,
                                                  conf_idx=args.conf_idx,
                                                  loc_len=args.loc_len,
                                                  od_seq_len=args.ob_seq_len,
                                                  moc_iou_token=SPC_TOKENS['<RT_OD_CONF_UNK>']
                                                 )

                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(),args.clip)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                save_counter += 1
                if save_counter % args.save_every == 0:
                    print(f'Saving checkpoint here {args.model_name}_rudolph_{save_counter}.pt')
                    plt.plot(loss_logs)
                    plt.show()
                    state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict(), 'losslogger': loss_logs}
                    torch.save(
                        model.state_dict(),
                        os.path.join(args.save_path,f"{args.model_name}rudolph_{save_counter}.pt")
                    )
                    torch.save(
                        state,
                        os.path.join(args.save_path,f"{args.model_name}state_{save_counter}.pt")
                    )

                loss_logs+=[loss.item()]
                progress.update()
                progress.set_postfix({"loss": loss.item()})

        print(f'Complitly tuned and saved here  {args.model_name}__object_detection_last.pt')
        plt.plot(loss_logs)
        plt.show()
        torch.save(
            model.state_dict(),
            os.path.join(args.save_path,f"{args.model_name}object_detection_last.pt")
        )

In [ ]:
model = freeze(
    model=model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False,
)

train(model, optimizer, scheduler, args, train_dataloader, from_checkpoints=False)

🦌🦌🦌finetuning process🦌🦌🦌:   0%|          | 0/419308 [00:00<?, ?it/s]